In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import necessary libraries

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import os
import glob
import json
import dask.dataframe as dd

# Read data

In [ ]:
# input_file_path = '/content/drive/MyDrive/Senzmate/RCA/results/jsonoutput3_cleaned_data.csv'

input_file_path = '/content/drive/MyDrive/Senzmate/RCA/results/test_jsonoutput3_cleaned_data.csv'

In [ ]:
main_df=pd.read_csv(input_file_path,  on_bad_lines='skip')

In [ ]:
main_df.head()

,deviceId,code,time,value
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10
2,869170034808734,B,2022-01-01T00:00:13.521Z,266
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55


# Check high volume data

In [ ]:
 main_df["high_volume_flag"] = main_df.duplicated(subset=['deviceId', 'time', 'code'], keep=False) 
 main_df.head()

,deviceId,code,time,value,high_volume_flag
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14,False
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10,False
2,869170034808734,B,2022-01-01T00:00:13.521Z,266,False
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44,False
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55,False


In [ ]:
main_df[main_df.high_volume_flag == True].shape[0]

3892

In [ ]:
main_df[main_df.high_volume_flag == False].shape[0]

109152

In [ ]:
main_df.shape

(113044, 5)

# Check missing data

In [ ]:
needed_sensors = ["B", "IT", "LIA1", "H", "IRO", "T", "ST", "SS"]

In [ ]:
grouped = main_df.groupby(['deviceId', 'time'])

In [ ]:
missing_data = []
for name, group in grouped:
  captured_sensors = group['code'].to_list()

  missing_sensors = list((Counter(needed_sensors) - Counter(captured_sensors)).elements())
  length_missing_sensors = len(missing_sensors)

  missing_df = pd.DataFrame({'deviceId' : [name[0]]*length_missing_sensors, 
                             'code' : missing_sensors,
                             'time' : name[1] * length_missing_sensors,
                             'value' : [0] * length_missing_sensors,
                             'defeat_flag' : [0] * length_missing_sensors
                             })

  missing_data.append(missing_df)

In [ ]:
missing_data_df = pd.concat(missing_data, ignore_index=True)
missing_data_df.head()

,deviceId,code,time,value,defeat_flag
0,869170034808734,IT,2022-01-01T00:00:13.521Z2022-01-01T00:00:13.52...,0,0
1,869170034808734,LIA1,2022-01-01T00:00:13.521Z2022-01-01T00:00:13.52...,0,0
2,869170034808734,IRO,2022-01-01T00:00:13.521Z2022-01-01T00:00:13.52...,0,0
3,869170034808734,ST,2022-01-01T00:00:13.521Z2022-01-01T00:00:13.52...,0,0
4,869170034808734,SS,2022-01-01T00:00:13.521Z2022-01-01T00:00:13.52...,0,0


In [ ]:
output_file_path = '/content/drive/MyDrive/Senzmate/RCA/results/test_jsonoutput3_missing_data.csv'
missing_data_df.to_csv(output_file_path, index=False) 

# Check sensor value range

In [ ]:
VALUES_RANGE = {'B': {"min": 260, "max": 314},
                'IT': {"min": -55, "max": 125},
                'LIA1': {"min": 0, "max": 65535},
                'H': {"min": 0, "max": 110},
                'IRO': {"min": 0, "max": 200},
                'T': {"min": -40, "max": 125},
                'ST': {"min": -55, "max": 125},
                'SS': {"min": 2, "max": 30},
                }
def detect_outlier(row):
  try:
    if row.high_volume_flag == True:
      return 3
    if float(row.value) < VALUES_RANGE[row.code]['min']:
      return 1
    if VALUES_RANGE[row.code]['max'] < float(row.value):
      return 2
  except:
    return 3
  return 4

In [ ]:
main_df['defeat_flag'] = main_df.apply(lambda row: detect_outlier(row), axis=1)
main_df.head()

,deviceId,code,time,value,high_volume_flag,defeat_flag
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14,False,4
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10,False,4
2,869170034808734,B,2022-01-01T00:00:13.521Z,266,False,4
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44,False,4
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55,False,4


# Remove high_volume_flag

In [ ]:
main_df.drop('high_volume_flag', axis=1, inplace=True)
main_df.head(5)

,deviceId,code,time,value,defeat_flag
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14,4
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10,4
2,869170034808734,B,2022-01-01T00:00:13.521Z,266,4
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44,4
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55,4


In [ ]:
output_file_path = '/content/drive/MyDrive/Senzmate/RCA/results/test_jsonoutput3_main_df.csv'
main_df.to_csv(output_file_path, index=False) 

In [ ]:
result_df = pd.concat([main_df, missing_data_df], ignore_index=True)

In [ ]:
result_df.head()

,deviceId,code,time,value,defeat_flag
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14,4
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10,4
2,869170034808734,B,2022-01-01T00:00:13.521Z,266,4
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44,4
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55,4


In [ ]:
result_df[result_df.defeat_flag == 0].shape[0]

147173

In [ ]:
result_df[result_df.defeat_flag == 1].shape[0]

322

In [ ]:
result_df[result_df.defeat_flag == 2].shape[0]

36

In [ ]:
result_df[result_df.defeat_flag == 3].shape[0]

4073

In [ ]:
result_df[result_df.defeat_flag == 4].shape[0]

108613

In [ ]:
output_file_path = '/content/drive/MyDrive/Senzmate/RCA/results/test_jsonoutput3_result.csv'
result_df.to_csv(output_file_path, index=False) 

In [ ]:
def convert_to_class(row):
  if row.defeat_flag == 0:
      return "MISSING"
  elif row.defeat_flag == 1:
      return "LOW_OUTLIER"
  elif row.defeat_flag == 2:
      return "HIGH_OUTLIER"
  elif row.defeat_flag == 3:
      return "UNKNOWN"
  return "NO_DEFEAT"

In [ ]:
result_df['defeat'] = result_df.apply(lambda row: convert_to_class(row), axis=1)
result_df.head()

,deviceId,code,time,value,defeat_flag,defeat
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14,4,NO_DEFEAT
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10,4,NO_DEFEAT
2,869170034808734,B,2022-01-01T00:00:13.521Z,266,4,NO_DEFEAT
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44,4,NO_DEFEAT
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55,4,NO_DEFEAT


In [ ]:
result_df.drop('defeat_flag', axis=1, inplace=True)
result_df.head(5)

,deviceId,code,time,value,defeat
0,869170034808734,T,2022-01-01T00:00:13.521Z,21.14,NO_DEFEAT
1,869170034808734,H,2022-01-01T00:00:13.521Z,103.10,NO_DEFEAT
2,869170034808734,B,2022-01-01T00:00:13.521Z,266,NO_DEFEAT
3,869170034809062,T,2022-01-01T00:00:16.445Z,13.44,NO_DEFEAT
4,869170034809062,H,2022-01-01T00:00:16.445Z,98.55,NO_DEFEAT


In [ ]:
output_file_path = '/content/drive/MyDrive/Senzmate/RCA/results/test_jsonoutput3_final.csv'
result_df.to_csv(output_file_path, index=False) 